In [2]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import eigs
import matplotlib.pyplot as plt

In [3]:


def create_laplacian_matrix(N, h):
    # Number of interior points
    n = N - 2
    
    # Main diagonal
    main_diag = -4 * np.ones(n * n)
    
    # Off-diagonals
    off_diag = np.ones(n * n)
    
    # Create the Laplacian matrix
    diagonals = [main_diag, off_diag, off_diag, off_diag, off_diag]
    offsets = [0, -1, 1, -n, n]
    A = diags(diagonals, offsets, shape=(n * n, n * n)).toarray()
    print(A)
    
    # Adjust for boundary conditions
    for i in range(1, n):
        A[i * n, i * n - 1] = 0
        A[i * n - 1, i * n] = 0
    
    return A / h**2

# Parameters
N = 6  # Number of grid points in each direction (including boundaries)
h = 1.0 / (N - 1)  # Grid spacing
# Create the Laplacian matrix
A = create_laplacian_matrix(N, h)
#print(A)

[[-4.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1. -4.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1. -4.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1. -4.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  1. -4.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  1. -4.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  1. -4.  1.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  1. -4.  1.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  1. -4.  1.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  1. -4.  1.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  1. -4.  1.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1. -4.  1.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1. -4.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1. -4.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1. -4. 